<a href="https://colab.research.google.com/github/Duckucy/113-2_Programing_Language/blob/main/HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 學生行為聚類分析（UCI Student Performance）

本專案使用 UCI Student Performance 資料集，透過 Google Sheet 載入，模擬學生的「觀看」與「完成」行為，並應用 KMeans 聚類與 PCA 降維視覺化，探索學生學習行為的分群樣貌。

## 📁 資料來源

- 原始資料集：[UCI Student Performance](https://archive.ics.uci.edu/dataset/320/student%2Bperformance)
- Google Sheet 連結：[https://docs.google.com/spreadsheets/d/1jKQYyliIozxJF43a-A-9TzGS0OSRZ4rGyYpemTJHvXE/edit](https://docs.google.com/spreadsheets/d/1jKQYyliIozxJF43a-A-9TzGS0OSRZ4rGyYpemTJHvXE/edit)

## 🔍 分析步驟

### 1. 載入資料

透過 Google Colab 授權，從 Google Sheet 讀取學生行為資料並轉為 DataFrame。

### 2. 資料前處理

- 將數值欄位（studytime、failures、absences、G1~G3）轉為 `int`
- 將類別欄位（schoolsup、famsup、paid、activities、higher、internet）轉為數值（yes → 1，no → 0）

### 3. 建立學習行為指標

- **觀看指標**：`studytime × 2 + activities × 3`
- **完成指標**：`G1 + G2 + G3 − failures × 2`

### 4. 聚類分析（KMeans）

使用學生的「觀看」與「完成」兩個行為指標，進行 KMeans 分群，設定群數為 4。

### 5. PCA 降維與視覺化

- 使用 PCA 將 2 維行為數據進一步轉為主成分（pca1、pca2）
- 使用 Plotly 畫出互動式散點圖，觀察學生群組分佈

## 📊 聚類結果解釋

| 群組 | 特徵說明 |
|------|----------|
| **群組 0** | 觀看與完成皆偏高，可能為積極學習者。 |
| **群組 1** | 完成高但觀看少，可能為自學能力強的學生。 |
| **群組 2** | 觀看高但完成低，可能為被動學習者或缺乏後續投入。 |
| **群組 3** | 觀看與完成都偏低，可能為低參與學生，需教師關注。 |

In [ ]:
!pip install scikit-learn

In [ ]:
# 學生行為聚類分析：使用 UCI Student Performance 資料集（從 Google Sheet 載入）

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import plotly.express as px

# Google Colab 認證與載入 Google Sheet
from google.colab import auth
import gspread
from google.auth import default

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# 讀取 UCI Student Performance 資料集
sheet_url = 'https://docs.google.com/spreadsheets/d/1jKQYyliIozxJF43a-A-9TzGS0OSRZ4rGyYpemTJHvXE/edit'
gsheet = gc.open_by_url(sheet_url)
worksheet = gsheet.sheet1  # 預設第一個工作表
sheets = worksheet.get_all_values()
df = pd.DataFrame(sheets[1:], columns=sheets[0])

# 轉換數值欄位
numeric_columns = ['studytime', 'failures', 'absences', 'G1', 'G2', 'G3']
for col in numeric_columns:
    df[col] = pd.to_numeric(df[col])

# 類別變數轉為數值
binary_map = {'yes': 1, 'no': 0}
for col in ['schoolsup', 'famsup', 'paid', 'activities', 'higher', 'internet']:
    df[col] = df[col].map(binary_map)

# 模擬學生觀看與完成行為
# 觀看：由 studytime 和是否參加活動模擬
# 完成：由三次成績相加減失敗次數模擬
df['觀看'] = df['studytime'] * 2 + df['activities'] * 3
df['完成'] = df['G1'] + df['G2'] + df['G3'] - df['failures'] * 2

# 進行標準化與 KMeans 聚類
behavior_data = df[['觀看', '完成']]
scaled_data = StandardScaler().fit_transform(behavior_data)

kmeans = KMeans(n_clusters=4, random_state=42)
df['cluster'] = kmeans.fit_predict(scaled_data)

# PCA 降維視覺化
pca = PCA(n_components=2)
components = pca.fit_transform(scaled_data)
df['pca1'] = components[:, 0]
df['pca2'] = components[:, 1]

# 繪製結果
fig = px.scatter(
    df,
    x='pca1',
    y='pca2',
    color='cluster',
    title="學生行為聚類結果 (從 Google Sheet 載入 UCI Student Performance)",
    labels={'pca1': '主成分1', 'pca2': '主成分2', 'cluster': '聚類群組'},
    hover_data=['studytime', 'failures', 'absences', 'G1', 'G2', 'G3', '觀看', '完成']
)
fig.show()
